# Clasificación binaria
Se implementará un modelo de clasificación binaria para un datasets con clases altamente desbalanceadas.

## Descargar el dataset

In [1]:
!curl -O https://storage.googleapis.com/naylalex-deep-learning/Kaggle-Credit-Card-Fraud-Detection.zip
!unzip -d datasets/credit-card-fraud-detection/ Kaggle-Credit-Card-Fraud-Detection.zip
!rm Kaggle-Credit-Card-Fraud-Detection.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 65.9M  100 65.9M    0     0  87.2M      0 --:--:-- --:--:-- --:--:-- 87.1M
Archive:  Kaggle-Credit-Card-Fraud-Detection.zip
  inflating: datasets/credit-card-fraud-detection/creditcard.csv  


## Vectorizar los datos CSV

In [2]:
import csv
import numpy as np

# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
fname = "./datasets/credit-card-fraud-detection/creditcard.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


## Preparar un dataset de validación

In [3]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


## Analizar el desbalance de clases en los targets

In [4]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


## Normalizar los datos empleando estadísticas del training set

In [5]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

## Generar un modelo de clasificación binaria

In [6]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

2022-03-22 04:30:52.293050: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-22 04:30:52.293811: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-22 04:30:52.366704: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               7936      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 139,777
Trainable params: 139,777
Non-trainable params: 0
__________________________________________________

## Entrenar el modelo con el argumento class_weight

In [7]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

2022-03-22 04:30:56.498667: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-22 04:30:56.519340: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2593905000 Hz


Epoch 1/30
112/112 - 11s - loss: 2.2235e-06 - fn: 40.0000 - fp: 26133.0000 - tn: 201296.0000 - tp: 377.0000 - precision: 0.0142 - recall: 0.9041 - val_loss: 0.0536 - val_fn: 11.0000 - val_fp: 435.0000 - val_tn: 56451.0000 - val_tp: 64.0000 - val_precision: 0.1283 - val_recall: 0.8533
Epoch 2/30
112/112 - 6s - loss: 1.5661e-06 - fn: 35.0000 - fp: 7696.0000 - tn: 219733.0000 - tp: 382.0000 - precision: 0.0473 - recall: 0.9161 - val_loss: 0.0700 - val_fn: 10.0000 - val_fp: 398.0000 - val_tn: 56488.0000 - val_tp: 65.0000 - val_precision: 0.1404 - val_recall: 0.8667
Epoch 3/30
112/112 - 6s - loss: 1.2005e-06 - fn: 32.0000 - fp: 6116.0000 - tn: 221313.0000 - tp: 385.0000 - precision: 0.0592 - recall: 0.9233 - val_loss: 0.1051 - val_fn: 7.0000 - val_fp: 1678.0000 - val_tn: 55208.0000 - val_tp: 68.0000 - val_precision: 0.0389 - val_recall: 0.9067
Epoch 4/30
112/112 - 6s - loss: 1.0635e-06 - fn: 24.0000 - fp: 7354.0000 - tn: 220075.0000 - tp: 393.0000 - precision: 0.0507 - recall: 0.9424 - val_

## Conclusiones
* Número de transacciones de validación.
* Número de casos identificados correctamente como fraudulentos.
* Número de transacciones fraudulentas faltantes.
* Costo de etiquetar 441 transacciones legítimas.
Recomendaciones: Colocar un peso mayor en la clase 1 para reflejar que los falsos negativos son más costosos que los falsos positivos.
